In [1]:
from helpers import get_prayer_time, get_london_break_fast_time

In [5]:
df = get_london_break_fast_time()
df.head()

,date,subuh,maghrib,break (early)
0,Thu 1 Apr,05:02,19:36,18:41
1,Fri 2 Apr,05:00,19:38,18:40
2,Sat 3 Apr,04:57,19:40,18:39
3,Sun 4 Apr,04:55,19:41,18:38
4,Mon 5 Apr,04:53,19:43,18:37


In [6]:
df.to_csv('early-maghrib.csv', index=False)

In [14]:
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup

MECCA_URL = 'https://www.muslimpro.com/en/find?country_code=SA&country_name=Saudi%20Arabia&city_name=Mecca&coordinates=21.3890824,39.8579118'
LONDON_URL = 'https://www.muslimpro.com/en/find?country_code=GB&country_name=United%20Kingdom&city_name=London&coordinates=51.5073509,-0.1277583'

def get_london_break_fast_time():
    mecca_df = get_prayer_time(MECCA_URL)
    london_df = get_prayer_time(LONDON_URL)

    mecca_fasting_diff = (mecca_df.maghrib.apply(pd.Timestamp) 
                          - mecca_df.subuh.apply(pd.Timestamp))

    early_break_fast_time = (london_df.subuh.apply(pd.Timestamp) 
                             + mecca_fasting_diff)
    early_break_fast_time = [datetime.strftime(time, '%H:%M') 
                             for time in early_break_fast_time]
    
    london_df['break (early)'] = early_break_fast_time
    return london_df

def get_prayer_time(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    
    table = soup.find('table', {'class': 'prayer-times'})
    table_content = table.find_all('tr')
    table_content = table_content[1:]
    
    dates = []
    subuhs = []
    maghribs = []

    today = datetime.today().day - 1

    for idx, row in enumerate(table_content):
        if idx < today:
            continue

        row_content = row.find_all('td')

        # Get date of the day
        date = process_text_data(row_content, 0)
        # date = date + ' 2021'
        # date = datetime.strptime(date, '%a %d %b %Y')
        dates.append(date)
        
        # Subuh prayer time
        subuh = process_text_data(row_content, 1)
        subuhs.append(subuh)
        
        # Maghrib prayer time
        maghrib = process_text_data(row_content, 5)
        maghribs.append(maghrib)
        
    df = pd.DataFrame(dict(date=dates, subuh=subuhs, maghrib=maghribs))
    return df

def process_text_data(content, for_col):
    res = content[for_col].find(text=True)
    res = str(res)
    return res

In [15]:
get_london_break_fast_time()

,date,subuh,maghrib,break (early)
0,Wed 14 Apr,04:33,19:58,18:29
1,Thu 15 Apr,04:31,20:00,18:27
2,Fri 16 Apr,04:29,20:01,18:28
3,Sat 17 Apr,04:26,20:03,18:26
4,Sun 18 Apr,04:23,20:05,18:25
5,Mon 19 Apr,04:21,20:06,18:24
6,Tue 20 Apr,04:19,20:08,18:23
7,Wed 21 Apr,04:16,20:10,18:22
8,Thu 22 Apr,04:14,20:11,18:21
9,Fri 23 Apr,04:12,20:13,18:21
